###### Татур А.А.
###### 27.09.2021
###### ver 0.1

###### Цели:
1. Реализовать скачивание и сохранение файлов с использованием BeautifulSoup И Requests
2. Измерить количество времени, которое требуется для этого процесса без участия Thread И Multiprocessing.
3. Повторить, используя Thread
4. Повторить, Используя Muliprocessing
5. Оценить время.
6. Подобрать оптимальное число потоков/процессов

Загружаем модули

In [1]:
import requests
from bs4 import BeautifulSoup
import os
import shutil
import time

###### Выполняем построение первичного списка изображений

In [4]:
list_url = ["https://lenta.ru/"] 
images = []
for url in list_url:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, features="html.parser")
    for img in soup.findAll('img'):
        images.append(img.get('src'))

###### Очищаем список файлов, сотавляем только картинки

In [6]:
img_list_cleaned=[]
for i in range(len(images)):
    if images[i]==None or images[i].startswith('//'):
        continue
    elif images[i].endswith('jpg'):
        img_list_cleaned.append(images[i])
len(img_list_cleaned)

133

###### Создаем фунцию для скачивания изображения. На вход принимает - ссылку, номер картинки и директорию, в которую происходит скачивание

In [7]:
def img_downl(url,pic_num,dir_name):
    # Начинаем загрузку.
        file_path=dir_name+'\\image'+str(pic_num)+'.jpg'
        img=requests.get(url)
        open(file_path,"wb").write(img.content)

##### Создаем папку для скачки без процессов и потоков. Если она существует, то будет перезаписана.

In [8]:
try: 
    os.mkdir('Images')
except FileExistsError:
    shutil.rmtree ('Images')
    os.mkdir ('Images')

###### Загружаем файлы без потоков и процессов=

In [9]:
# Начинаем загрузку.
start=time.time()
for i in img_list_cleaned:
    img_downl(i,img_list_cleaned.index(i),'Images')
end = time.time()
# Оканчиваем загрузку.
print ("На загрузку потрачено - ", end-start, ' секунд')
print ("Скачано - ", len(img_list_cleaned), " файлов")

На загрузку потрачено -  17.865105628967285  секунд
Скачано -  133  файлов


######  Создаем папку для загрузки с потоками

In [10]:
try: 
    os.mkdir('ImagesThreads')
except FileExistsError:
    shutil.rmtree ('ImagesThreads')
    os.mkdir ('ImagesThreads')

######  Проводим загрузку с потоками. Особой разницы при игре с количеством потоков не заметил,
######  но сильно усложняется код из-за необходимости дробить список ссылок, что также приводит к 
###### потере файлов

In [11]:
import threading

start_time=time.time()

threads=[]

for i in img_list_cleaned:
    t1= threading.Thread(target=img_downl, args=[i, img_list_cleaned.index(i),'ImagesThreads'])
    t1.start()
    threads.append(t1)    
    
for thread in threads:
    thread.join()


end_time=time.time()
exec_time= end_time-start_time
print("На загрузку потрачено - ", end_time-start_time, ' секунд')

На загрузку потрачено -  1.292853832244873  секунд


######  Создаем папку для загрузки с мультипроцессами

In [12]:
try: 
    os.mkdir('ImagesMulti')
except FileExistsError:
    shutil.rmtree ('ImagesMulti')
    os.mkdir ('ImagesMulti')

######  Старт загрузки. Оптимально 8 процессов

In [ ]:
from multiprocessing import Pool, cpu_count


# Создал отдельную функцию,т.к. с мультипроцессами получилось их запустить только с помощью функции pool.map. Поэтому упростил
# до одного аргумента

def img_downl_mult(url):
        file_path='ImagesMulti\\image'+str(img_list_cleaned.index(url))+'.jpg'
        img=requests.get(url)
        open(file_path,"wb").write(img.content)

if __name__ == '__main__':
    pool = Pool(8)
    start = time.time()
    results = pool.map(img_downl_mult, img_list_cleaned)
    pool.close()
    pool.join()
    end = time.time()
    print('Time taken in seconds -', end - start)